In [75]:
import numpy as np
import math
import itertools
import random
import matplotlib as mpl
import matplotlib.pyplot as plt
import qiskit
import qiskit.quantum_info as qi 
# import warnings
# warnings.filterwarnings('ignore')
import time
import networkx as nx
from scipy.linalg import expm
from scipy.optimize import minimize, Bounds
from qiskit import QuantumCircuit, Aer, execute


In [76]:
def get_circuit_operators( lst_x=None, lst_z=None):
    '''
    note the order! (x,z)
    '''
    # returns for example X @ I @ I @ X @ I where @ is tensor product
    return  qi.Pauli((lst_z,lst_x)).to_matrix()

In [170]:
def generate_QAOA_operator(H1,H2, number_of_layers = 1 , beta_angle = [], gamma_angle = []):
    """_summary_

    Args:
        H (_type_): _description_
        number_of_layers (int, optional): _description_. Defaults to 1.
        beta_angle (list, optional): _description_. Defaults to [].
        gamma_angle (list, optional): _description_. Defaults to [].

    Returns:
        _type_: _description_
    """    '''
    '''
    # define angles for mixers 
    if len(beta_angle)==0:
        beta_angle = np.random.rand(number_of_layers) * np.pi


    if len(gamma_angle)==0:
        gamma_angle = np.random.rand(number_of_layers) * np.pi

    ans = H1

    x_string = get_circuit_operators(np.ones(int(np.log2(len(H1)))),np.zeros(int(np.log2(len(H1)))))

    for p in range(number_of_layers):
        # Making unitary for Hamiltonian exp
        exp_H = expm(1j*H2*beta_angle[p])
        # Making mixer X
        exp_X = expm(1j*x_string*gamma_angle[p])

        ans = exp_H @ exp_X @ ans @ exp_X.T.conjugate() @ exp_H.T.conjugate()

    return ans
   

In [144]:
# produce all binary strings of length n with k 1s. If k is None then all possible binary strings of length n produced
def get_binary_strings(n, k=None) -> list[list]:
    '''
    produce all binary strings of length n with k 1s
    returns list with binary lists 
    '''
    final = []
    def kbits(r):
        result = []
        for bits in itertools.combinations(range(n), r):
            s = [0] * n
            for bit in bits:
                s[bit] = 1
            result.append(s)   
        return result

    if k != None:
        return kbits(k)
    
    for i in range(n + 1):
        final = final + kbits(i)
        
    return final


In [145]:
def count_solutions_XI(nqubits: int, Operator: np.array, tol=1e-10, if_print = False ) -> tuple:
    
    x_strings = get_binary_strings(nqubits)
    z_zeros = np.zeros(nqubits)

    ans = []
    max_locality  = 0
    avg_locality = 0.0

    for i in x_strings:
        x_mat = get_circuit_operators(i,z_zeros)
        coef = 1/(1<<nqubits) * np.trace(x_mat@Operator)
        if np.abs(coef)>tol:
            count_x = np.sum(i)
            ans.append((str(i),coef))
            if count_x > max_locality:
                max_locality = count_x
            avg_locality += count_x
    len_ans = len(ans)
    if len_ans == 0:
        len_ans = 1
        print("len_ans is zero")
    
    if if_print:
        print("Non-zero Pauli strings:", len_ans)
        print("Max locality:", max_locality)
        print("Avg locality:", avg_locality/len_ans)
    return ans, max_locality, avg_locality/len_ans, len_ans


In [146]:
from itertools import product
from functools import reduce

PAULIS = {'I': np.eye(2),
          'X': np.array([[0, 1], [1, 0]]),
          'Y': np.array([[0, -1j], [1j, 0]]),
          'Z': np.diag([1, -1])}

def get_pauli_decomp(in_matrix, display_every=5000, PAULIS=PAULIS):
    """Return dictionary with pauli strings and according weight.
    in_matrix - input matrix (should be of size(2^m,2^m), where m is int.
    display_time - if given int number print time spend for computation if False do not print.
    PAULIS - dictionary with Pauli basis.

    """
    m = int(np.log2(in_matrix.shape[0]))

    pauli_weights = {}

    k = 1
    K = len(list(product(PAULIS.keys(), repeat=m)))

    start_ = time.time()
    for u in product(PAULIS.keys(), repeat=m):
        pauli_str_name = ''.join(u)
        pauli_str_matrix = reduce(np.kron, [PAULIS[s] for s in u])
        inner_product = np.trace(in_matrix @ pauli_str_matrix) / 2**m
        if not np.isclose(inner_product, 0):
            pauli_weights[pauli_str_name] = inner_product
        if display_every and k%display_every == 0:
            print('\t {} qubits || {}/{} || {:.3f} s passed'.format(m,k,K, time.time()-start_))
        k+=1

    return pauli_weights

In [147]:
N = 6
# H = np.diag(np.random.rand(2**N))
# generate_QAOA_operator(H)

In [148]:
rng = np.random.default_rng()

In [200]:
Zs = get_binary_strings(N,2)
H1 = 0
buf = rng.choice(Zs, 5, replace=False)
for i in buf:
    H1 = H1 + get_circuit_operators(np.zeros(N),i)
H1

array([[ 5.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  1.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
       ...,
       [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  1.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j, -1.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  5.+0.j]])

In [201]:
H2 = np.copy(H1)
for i in rng.choice( get_binary_strings(N,3), 10, replace=False):
    H2 = H2 + get_circuit_operators(np.zeros(N),i)
H2

array([[15.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
       ...,
       [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  3.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j, -1.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j, -5.+0.j]])

In [202]:
get_pauli_decomp(generate_QAOA_operator(H1,H1))

{'IIIIZZ': (1+0j),
 'IIIZIZ': (1+0j),
 'IIIZZI': (1+0j),
 'IZIIIZ': (1+0j),
 'ZIZIII': (1+0j)}

In [168]:
count_solutions_XI(N,generate_QAOA_operator(H1,H2))

len_ans is zero


([], 0, 0.0, 1)

In [78]:
def make_H_maxCUT(n, adj_mat):
    H = np.zeros((2**n,2**n),dtype=complex)
    zeros = np.zeros(n)

    for i in range(n):
        for j in range(i+1,n):
            if adj_mat[i][j]!=0:
                buf = np.zeros(n)
                buf[i] = buf[j] = 1
                H += adj_mat[i][j]*get_circuit_operators(zeros, buf)
    return H

In [79]:
def plus_state(n):
    return 1/np.sqrt(2**n) * np.ones(2**n)

In [82]:
def output_state(theta, H, layer):
    n = int(np.log2(len(H)))
    circ = generate_QAOA_operator(H, number_of_layers = layer , beta_angle = theta[:len(theta)//2], gamma_angle = theta[len(theta)//2:])
    return np.dot(plus_state(n).T,circ)


def energy(theta, H, layer):
    state = output_state(theta, H, layer)
    n = int(np.log2(len(H)))
    return np.real(np.dot(state.conjugate().T, np.dot(H, state)))


def get_expectation(H, layer):

    def execute_circ(theta):
        return energy(theta, H, layer)

    return execute_circ


In [83]:
def theta_for_ground_state(H,layer=1, repeat=10, maxiter=100):
    bds = Bounds(0, 2 * np.pi)
    expectation = get_expectation(H,layer)
    for i in range(repeat):
        initial_params = np.random.uniform(0, np.pi, layer*2)  # initial guess of parameters
        res = minimize(expectation,
                       initial_params, method='L-BFGS-B', jac='3-point', bounds=bds, options={'maxiter': maxiter})
        if i == 0:
            E_min = res.fun
            params = res.x
        if res.fun < E_min:
            E_min = res.fun
            params = res.x
        # print(E_min)
    return params, E_min

In [85]:
H = make_H_maxCUT(2, [[0,1],[1,0]])
H

array([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j]])

In [86]:
theta_for_ground_state(H)

(array([2.55498539, 1.73577337]), -4.940986256670209e-34)

In [87]:
output_state([2.55498539, 1.73577337],H,1)

array([ 0.5+1.19313693e-16j, -0.5-1.19313693e-16j, -0.5+1.74824845e-16j,
        0.5-6.38025421e-17j])